In [ ]:
import os
import numpy as np
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

In [21]:
def simulate_sphere_clustering(
    n: int = 20,
    d: int = 3,
    beta: float = 0.0,
    dt: float = 0.01,
    steps: int = 1000,
    seed: int = 0,
    antipodal_pair: bool = False,
    force_2d: bool = False,
    save_mp4: bool = False,
    gif_dir: str = "../plots",
):
    """
    Simulates a clustering process on the unit sphere using a gradient-like update.
    """

    if d < 2:
        raise ValueError("Dimension d must be >= 2.")

    np.random.seed(seed)
    x = np.random.randn(n, d)
    x = x / np.linalg.norm(x, axis=1, keepdims=True)

    if antipodal_pair:
        base = np.random.randn(d)
        base = base / np.linalg.norm(base)
        x[0] = base
        x[1] = -base
        x = x[:2]
        n = 2

    def project(xi, y):
        return y - np.dot(xi, y) * xi

    trajectory = np.zeros((steps, n, d))
    trajectory[0] = x.copy()

    for step in range(1, steps):
        x_new = np.zeros((n, d))
        for i in range(n):
            dots = np.dot(x[i], x.T)
            weights = np.exp(beta * dots)
            Z = np.sum(weights)
            if not np.isfinite(Z) or Z == 0:
                raise ValueError(
                    f"Invalid normalization factor Z={Z} at step={step}, particle={i}"
                )
            weighted_sum = np.sum(weights[:, np.newaxis] * x, axis=0) / Z
            delta = project(x[i], weighted_sum)
            xi_new = x[i] + dt * delta
            x_new[i] = xi_new / np.linalg.norm(xi_new)
        x = x_new
        trajectory[step] = x

    def plot_2d_projection(ax, x_step, step):
        ax.clear()
        ax.add_artist(plt.Circle((0, 0), 1, color="gray", fill=False))
        ax.set_aspect("equal")
        if d > 2:
            sc = ax.scatter(
                x_step[:, 0], x_step[:, 1], c=x_step[:, 2], cmap="coolwarm", s=50
            )
        else:
            sc = ax.scatter(x_step[:, 0], x_step[:, 1], color="blue", s=50)
        ax.set_xlim([-1.2, 1.2])
        ax.set_ylim([-1.2, 1.2])
        ax.set_title(f"Step {step + 1} | n={n}, d={d}, β={beta}")
        return sc

    def plot_3d(ax, x_step, step):
        ax.clear()
        ax.scatter(x_step[:, 0], x_step[:, 1], x_step[:, 2], color="blue", s=50)
        u, v = np.linspace(0, 2 * np.pi, 30), np.linspace(0, np.pi, 15)
        xs = np.outer(np.cos(u), np.sin(v))
        ys = np.outer(np.sin(u), np.sin(v))
        zs = np.outer(np.ones_like(u), np.cos(v))
        ax.plot_wireframe(xs, ys, zs, color="gray", alpha=0.3, linewidth=0.5)
        ax.set_xlim([-1.2, 1.2])
        ax.set_ylim([-1.2, 1.2])
        ax.set_zlim([-1.2, 1.2])
        ax.set_box_aspect([1, 1, 1])
        ax.view_init(elev=25, azim=45)
        ax.grid(False)
        ax.set_title(f"Step {step + 1} | n={n}, d={d}, β={beta}")

    if save_mp4:
        os.makedirs(gif_dir, exist_ok=True)
        extra_info = ""
        extra_info += f"_force2d={force_2d}" if force_2d else ""
        extra_info += f"_antipodal={antipodal_pair}" if antipodal_pair else ""
        filename = f"clustering_n={n}_d={d}_beta={beta}_dt={dt}_steps={steps}_seed={seed}{extra_info}.mp4"
        filepath = os.path.join(gif_dir, filename)
        
        fig = plt.figure(figsize=(6, 6))
        ax = (
            fig.add_subplot(111, projection="3d")
            if d == 3 and not force_2d
            else fig.add_subplot(111)
        )

        def update(frame):
            x_step = trajectory[frame]
            if d == 3 and not force_2d:
                plot_3d(ax, x_step, frame)
            else:
                plot_2d_projection(ax, x_step, frame)
            return (ax,)

        ani = animation.FuncAnimation(
            fig, update, frames=range(0, steps, max(1, steps // 100)), blit=False, repeat=False
        )
        
        ani.save(filepath, writer='ffmpeg', fps=15, dpi=200)
        plt.close()

    # Interactive visualization
    def plot_step(step):
        step = step - 1
        x_step = trajectory[step]
        if d == 3 and not force_2d:
            fig = plt.figure(figsize=(6, 6))
            ax = fig.add_subplot(111, projection="3d")
            plot_3d(ax, x_step, step)
            plt.show()
        else:
            fig = plt.figure(figsize=(5, 5))
            ax = fig.add_subplot(111)
            plot_2d_projection(ax, x_step, step)
            plt.show()

    slider = widgets.IntSlider(min=1, max=steps, step=10, value=1, description="Step")
    widgets.interact(plot_step, step=slider)


In [32]:
simulate_sphere_clustering(
    n=50,
    d=2,
    beta=2.4,
    steps=1000,
    seed=2025,
    # antipodal_pair=True
    # force_2d=True
    save_mp4=True,
)

interactive(children=(IntSlider(value=1, description='Step', max=1000, min=1, step=10), Output()), _dom_classe…